In [6]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import sys
import time

from moviepy.editor import VideoFileClip

from IPython.display import HTML

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

from skimage.feature import hog

from scipy.ndimage.measurements import label

%matplotlib inline

In [16]:
def pipeline(img):

    output = np.zeros((1080, 1800, 3), dtype=np.uint8)
    
    output[360:1080, 0:1200] = cv2.resize(img, (1200,720), interpolation=cv2.INTER_AREA) 

    output[0:360, 0:600] = cv2.resize(img, (600,360), interpolation=cv2.INTER_AREA)   

    output[0:360, 600:1200] = cv2.resize(img, (600,360), interpolation=cv2.INTER_AREA)

    output[0:360, 1200:1800] = cv2.resize(img, (600,360), interpolation=cv2.INTER_AREA)

    output[360:720,1200:1800] = cv2.resize(img, (600,360), interpolation=cv2.INTER_AREA) 

    output[720:1080,1200:1800] = cv2.resize(img, (600,360), interpolation=cv2.INTER_AREA) 

    #output[360:720,1080:1440] = cv2.resize(np.dstack((perspective*255, perspective*255, perspective*255)), (360,360), interpolation=cv2.INTER_AREA) 

    #output[720:1080,1080:1440] = cv2.resize(lane, (360,360), interpolation=cv2.INTER_AREA) 

    return output

In [17]:
white_output = 'result.mp4'
clip1 = VideoFileClip("test_video.mp4")
white_clip = clip1.fl_image(pipeline) 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video result.mp4
[MoviePy] Writing video result.mp4


 97%|█████████▋| 38/39 [00:05<00:00,  6.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: result.mp4 

CPU times: user 12.3 s, sys: 526 ms, total: 12.9 s
Wall time: 10.1 s


In [18]:
HTML("""
<video width="900" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))